In [1]:
import os
import gc
import numpy as np
import pandas as pd

In [2]:
# メモリ使用量の削減
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def read_data():
    print('Reading files...')

    calendar_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')
    calendar_df = reduce_mem_usage(calendar_df)
    print('Calendar: ' + str(calendar_df.shape))
    print("{0} information in {1} days".format(calendar_df.shape[1], calendar_df.shape[0]))

    sell_prices_df = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
    sell_prices_df = reduce_mem_usage(sell_prices_df)
    print('Sell prices: ' + str(sell_prices_df.shape))

    train_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
    print('Sales train validation: ' + str(train_df.shape))

    submission_df = pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv')
    print("Submission: " + str(submission_df.shape))
    return calendar_df, sell_prices_df, train_df, submission_df

In [4]:
def melt_and_merge(calendar_df, sell_prices_df, train_df, submission_df):

    # 商品情報を抽出
    product_df = train_df.loc[:, "id":"state_id"]
    
    # 列方向に連なっていたのを変形し行方向に連ねるように整理
    train_df = pd.melt(train_df, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                       var_name = 'day', value_name = 'demand')
    train_df = reduce_mem_usage(train_df)
    train_day = train_df["day"].unique()
    print("\ntrain_data: {} ~ {}".format(train_day[0], train_day[-1]))

    # seperate test dataframes
    valid_df = submission_df[submission_df["id"].str.contains("validation")]
    eval_df = submission_df[submission_df["id"].str.contains("evaluation")]
    
    # change column names
    valid_df.columns = ["id"] + [f"d_{d}" for d in range(1914, 1942)]  # validation data: F1 ~ F28 => d_1914 ~ d_1941
    eval_df.columns = ["id"] + [f"d_{d}" for d in range(1942, 1970)]  # evaluation data: F1 ~ F28 => d_1942 ~ d_1969

    # melt, mergeを使ってsubmission用のdataframeを上のsales_train_validationと同様の形式に変形
    valid_df = valid_df.merge(product_df, how = 'left', on = 'id')
    valid_df = pd.melt(valid_df, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                       var_name = 'day', value_name = 'demand')
    valid_day = valid_df["day"].unique()
    print("valid_data[STAGE1]: {} ~ {}".format(valid_day[0], valid_day[-1]))

    # train_df, valid_dfと同様にeval_dfとproduct_dfをmergeさせたい
    # しかしidが_evaluationのままだとデータが一致せずmergeできないので一時的に_validationにidを変更
    eval_df['id'] = eval_df.loc[:, 'id'].str.replace('_evaluation','_validation')
    eval_df = eval_df.merge(product_df, how = 'left', on = 'id')
    eval_df['id'] = eval_df.loc[:, 'id'].str.replace('_validation','_evaluation')
    eval_df = pd.melt(eval_df, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                      var_name = 'day', value_name = 'demand')
    eval_day = eval_df["day"].unique()
    print("eval_data[STAGE2]: {} ~ {}".format(eval_day[0], eval_day[-1]))

    train_df['part'] = 'train'
    valid_df['part'] = 'valid'
    eval_df['part'] = 'eval'
    
    data_df = pd.concat([train_df, valid_df, eval_df], axis = 0)
    print("\n[INFO] data_df(after merge valid & eval) ->")
    print(data_df.head(5))
    print(data_df.columns)

    # 不要なdataframeの削除
    del train_df, valid_df, eval_df
    gc.collect()
    
    # NOTE get only a sample for fast training
    # data_df = data_df.loc[nrows:]
    # print("\n[CHECK] Remove some train data")
    
    # drop some calendar features
    calendar_df.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete eval_df for now
    data_df = data_df[data_df['part'] != 'eval']
    print("[CHECK] Remove the eval data in {} ~ {}".format(eval_day[0], eval_day[-1]))

    # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
    data_df = pd.merge(data_df, calendar_df, how = 'left', left_on = ['day'], right_on = ['d'])
    data_df.drop(['d', 'day'], inplace = True, axis = 1)

    # get the sell price data (this feature should be very important)
    data_df = data_df.merge(sell_prices_df, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
    print("\n[INFO] data_df(after merge calendar & prices) ->")
    print(data_df.head(5))
    print(data_df.columns)

    del calendar_df, sell_prices_df
    gc.collect()
    
    return data_df, product_df

In [5]:
calendar_df, sell_prices_df, train_df, submission_df = read_data()

Reading files...
Calendar: (1969, 14)
14 information in 1969 days
Sell prices: (6841121, 4)
Sales train validation: (30490, 1919)
Submission: (60980, 29)


In [6]:
data_df, product_df = melt_and_merge(calendar_df, sell_prices_df, train_df, submission_df)


train_data: d_1 ~ d_1913
valid_data[STAGE1]: d_1914 ~ d_1941
eval_data[STAGE2]: d_1942 ~ d_1969

[INFO] data_df(after merge valid & eval) ->
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  day  demand   part  
0       CA  d_1       0  train  
1       CA  d_1       0  train  
2       CA  d_1       0  train  
3       CA  d_1       0  train  
4       CA  d_1       0  train  
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day',
       'demand', 'part'],
      dtype='object')
[CHECK] Remove the eval data in d_1942 ~ d_1969

[INFO]

In [7]:
product_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA


In [8]:
product_df.shape

(30490, 6)

30490個の商品情報(店舗で分けているので被りはある)

In [9]:
data_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN


In [10]:
print("train & valid:{}".format(data_df.shape))
print("train:{}".format(data_df[data_df["part"]=="train"].shape))
print("valid:{}".format(data_df[data_df["part"]=="valid"].shape))

train & valid:(59181090, 18)
train:(58327370, 18)
valid:(853720, 18)


In [11]:
train_df = data_df[data_df["part"]=="train"]
valid_df = data_df[data_df["part"]=="valid"]

In [12]:
train_df["demand"].unique()

array([  0,  12,   2,   4,   5,   9,   3,   1,  25,  45,  10,   8,   7,
        16,  54,   6,  50,  14,  11,  17,  15,  23,  13,  20,  19,  38,
        32,  21,  30,  33, 107,  51,  26, 147,  22,  43,  24,  35,  27,
        42,  49,  36,  18,  37, 116,  56,  28,  34,  41,  31,  39,  44,
       108,  55, 360,  62,  69,  75, 125,  53,  47,  29, 164,  52, 102,
        63,  98,  59, 203, 152,  65, 115,  46,  57,  90,  58,  78,  64,
       130,  60, 208,  93,  61,  40,  73, 182, 198, 121, 132, 436, 123,
       215, 160,  85, 112, 141,  79,  48, 153, 204,  70,  67,  92,  86,
       207,  74,  94,  66,  91, 128,  84,  71,  83,  72, 119,  76,  99,
       134,  80,  89, 106, 120, 323, 127, 139, 109, 140, 114, 296, 314,
       159,  96,  82,  81, 180, 137, 171, 100, 279, 316, 150,  97, 169,
        68,  87, 174, 188, 195, 124, 202, 175, 370, 118, 200, 136, 187,
       250, 104, 367, 178,  95,  88, 214, 186, 385, 105,  77, 177, 184,
       162, 103, 228, 331, 353, 101, 154, 126, 258, 117, 151, 16

In [13]:
valid_df["demand"].unique()

array([0])

validデータ([STAGE1]の予測対象)はもちろんlabelはついてないので全て0を示している

In [14]:
for column in list(train_df.columns):
    print("\n{}:{}".format(column, train_df[column].nunique()))
    # print(train_df[column].unique())


id:30490

item_id:3049

dept_id:7

cat_id:3

store_id:10

state_id:3

demand:419

part:1

date:1913

wm_yr_wk:274

event_name_1:30

event_type_1:4

event_name_2:4

event_type_2:2

snap_CA:2

snap_TX:2

snap_WI:2

sell_price:1026


In [15]:
train_df["event_name_1"].unique()

array([nan, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)

In [16]:
train_df[train_df["event_name_2"] == "Father's day"]["event_name_1"].unique()

array(['NBAFinalsEnd'], dtype=object)

event_2はevent_1に含まれているため現状あまり重要ではない

validデータと同時期の異なる年のデータを見たい(04/25~05/22)  
同じ傾向にあるのか   
近さが大事なのか  
時系列が大事なのか  
周期性が大事なのか 

In [17]:
valid_df["date"].unique()

array(['2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
       '2016-04-29', '2016-04-30', '2016-05-01', '2016-05-02',
       '2016-05-03', '2016-05-04', '2016-05-05', '2016-05-06',
       '2016-05-07', '2016-05-08', '2016-05-09', '2016-05-10',
       '2016-05-11', '2016-05-12', '2016-05-13', '2016-05-14',
       '2016-05-15', '2016-05-16', '2016-05-17', '2016-05-18',
       '2016-05-19', '2016-05-20', '2016-05-21', '2016-05-22'],
      dtype=object)

In [18]:
train2011 = train_df[(train_df["date"] > '2011-04-24') & (train_df["date"] < '2011-05-23')]
train2012 = train_df[(train_df["date"] > '2012-04-24') & (train_df["date"] < '2012-05-23')]
train2013 = train_df[(train_df["date"] > '2013-04-24') & (train_df["date"] < '2013-05-23')]
train2014 = train_df[(train_df["date"] > '2014-04-24') & (train_df["date"] < '2014-05-23')]
train2015 = train_df[(train_df["date"] > '2015-04-24') & (train_df["date"] < '2015-05-23')]

In [19]:
train2011.shape

(853720, 18)

In [20]:
train2011.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
2622140,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-04-25,11113,NaN,NaN,NaN,NaN,0,0,0,NaN
2622141,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-04-25,11113,NaN,NaN,NaN,NaN,0,0,0,NaN
2622142,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-04-25,11113,NaN,NaN,NaN,NaN,0,0,0,NaN
2622143,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-04-25,11113,NaN,NaN,NaN,NaN,0,0,0,4.339844
2622144,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-04-25,11113,NaN,NaN,NaN,NaN,0,0,0,NaN


In [21]:
train2011["dept_id"].unique()

array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
       'FOODS_2', 'FOODS_3'], dtype=object)

In [22]:
import matplotlib.pyplot as plt
dept_id = train2011["dept_id"].unique()

for i in range(len(dept_id)):
    fig = plt.figure(figsize = (20, 5))
    ax1 = fig.add_subplot(2, 7, 2(i+1)-1)
    ax2 = fig.add_subplot(2, 7, 2(i+1))

    ax1.plot(train2011[train2011["dept_id"]=='HOBBIES_1'].groupby("date").mean()["sell_price"].values, label='2011')
    ax1.plot(train2012[train2012["dept_id"]=='HOBBIES_1'].groupby("date").mean()["sell_price"].values, label='2012')
    ax1.plot(train2013[train2013["dept_id"]=='HOBBIES_1'].groupby("date").mean()["sell_price"].values, label='2013')
    ax1.plot(train2014[train2014["dept_id"]=='HOBBIES_1'].groupby("date").mean()["sell_price"].values, label='2014')
    ax1.plot(train2015[train2015["dept_id"]=='HOBBIES_1'].groupby("date").mean()["sell_price"].values, label='2015')
    ax1.set_title("{} sell prices".format(dept_id[i]))
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Sales')
    ax1.legend()

    ax2.plot(train2011[train2011["dept_id"]=='HOBBIES_1'].groupby("date").mean()["demand"].values, label='2011')
    ax2.plot(train2012[train2012["dept_id"]=='HOBBIES_1'].groupby("date").mean()["demand"].values, label='2012')
    ax2.plot(train2013[train2013["dept_id"]=='HOBBIES_1'].groupby("date").mean()["demand"].values, label='2013')
    ax2.plot(train2014[train2014["dept_id"]=='HOBBIES_1'].groupby("date").mean()["demand"].values, label='2014')
    ax2.plot(train2015[train2015["dept_id"]=='HOBBIES_1'].groupby("date").mean()["demand"].values, label='2015')
    ax2.set_title("{} demand".format(dept_id[i]))
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Sales')
    ax2.legend()

TypeError: 'int' object is not callable

In [28]:
data_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,black_friday
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0


In [29]:
# black fridayの追加
black_friday = ["2011-11-25", "2012-11-23", "2013-11-29", "2014-11-28", "2015-11-27"]
data_df["black_friday"] = data_df["date"].isin(black_friday) * 1
data_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,black_friday
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,0


sell_priceの各idごとの平均を算出しそこからsell_priceの差分を取ることで割引額、割引率を算出  
割引なので平均だけでなく、ちょっと前の時期の価格との差分を取るのもいいかも  

In [30]:
data_df[data_df["item_id"] == "HOBBIES_1_001"]["sell_price"]

0                NaN
3049             NaN
6098             NaN
9147             NaN
12196            NaN
15245            NaN
18294            NaN
21343            NaN
24392            NaN
27441            NaN
30490            NaN
33539            NaN
36588            NaN
39637            NaN
42686            NaN
45735            NaN
48784            NaN
51833            NaN
54882            NaN
57931            NaN
60980            NaN
64029            NaN
67078            NaN
70127            NaN
73176            NaN
76225            NaN
79274            NaN
82323            NaN
85372            NaN
88421            NaN
              ...   
59089620    8.382812
59092669    8.382812
59095718    8.382812
59098767    8.382812
59101816    8.257812
59104865    8.257812
59107914    8.257812
59110963    8.382812
59114012    8.382812
59117061    8.382812
59120110    8.382812
59123159    8.382812
59126208    8.382812
59129257    8.382812
59132306    8.257812
59135355    8.257812
59138404    8

In [31]:
data_df.groupby("id").mean()

,demand,wm_yr_wk,snap_CA,snap_TX,snap_WI,sell_price,black_friday
id,,,,,,,
FOODS_1_001_CA_1_validation,0.774343,11343.166924,0.329727,0.329727,0.329727,2.167969,0.002576
FOODS_1_001_CA_2_validation,1.140649,11343.166924,0.329727,0.329727,0.329727,2.167969,0.002576
FOODS_1_001_CA_3_validation,1.185987,11343.166924,0.329727,0.329727,0.329727,2.156250,0.002576
FOODS_1_001_CA_4_validation,0.354456,11343.166924,0.329727,0.329727,0.329727,2.167969,0.002576
FOODS_1_001_TX_1_validation,0.576507,11343.166924,0.329727,0.329727,0.329727,2.156250,0.002576
FOODS_1_001_TX_2_validation,0.572901,11343.166924,0.329727,0.329727,0.329727,2.167969,0.002576
FOODS_1_001_TX_3_validation,0.404431,11343.166924,0.329727,0.329727,0.329727,2.167969,0.002576
FOODS_1_001_WI_1_validation,0.559505,11343.166924,0.329727,0.329727,0.329727,2.167969,0.002576
FOODS_1_001_WI_2_validation,0.450799,11343.166924,0.329727,0.329727,0.329727,2.167969,0.002576


In [32]:
print(data_df[data_df["id"] == "HOBBIES_1_001_CA_1_validation"]["sell_price"].mean())
print(data_df[data_df["id"] == "HOBBIES_1_001_CA_1_validation"]["sell_price"].dropna().mean())

8.28
8.28


gropbyで求めたsell_priceの平均値はNanを無視しているため気にしなくていい  
mean_sell_priceという列を作る  
欠損値がない部分はsell_priceとmeanとの差分を取る  
最初に全シーズンで平均出したら１月単位とかで平均出すのもあり  

In [8]:
mean_sell_price_df = data_df.groupby('id').mean()

In [9]:
mean_sell_price_df.rename(columns={"sell_price": "mean_sell_price"}, inplace=True)

In [13]:
mean_sell_price_df.head()

,demand,wm_yr_wk,snap_CA,snap_TX,snap_WI,mean_sell_price
id,,,,,,
FOODS_1_001_CA_1_validation,0.774343,11343.166924,0.329727,0.329727,0.329727,2.167969
FOODS_1_001_CA_2_validation,1.140649,11343.166924,0.329727,0.329727,0.329727,2.167969
FOODS_1_001_CA_3_validation,1.185987,11343.166924,0.329727,0.329727,0.329727,2.156250
FOODS_1_001_CA_4_validation,0.354456,11343.166924,0.329727,0.329727,0.329727,2.167969
FOODS_1_001_TX_1_validation,0.576507,11343.166924,0.329727,0.329727,0.329727,2.156250


In [14]:
mean_sell_price_df["mean_sell_price"]

id
FOODS_1_001_CA_1_validation         2.167969
FOODS_1_001_CA_2_validation         2.167969
FOODS_1_001_CA_3_validation         2.156250
FOODS_1_001_CA_4_validation         2.167969
FOODS_1_001_TX_1_validation         2.156250
FOODS_1_001_TX_2_validation         2.167969
FOODS_1_001_TX_3_validation         2.167969
FOODS_1_001_WI_1_validation         2.167969
FOODS_1_001_WI_2_validation         2.167969
FOODS_1_001_WI_3_validation         2.166016
FOODS_1_002_CA_1_validation         8.929688
FOODS_1_002_CA_2_validation         8.851562
FOODS_1_002_CA_3_validation         8.929688
FOODS_1_002_CA_4_validation         8.929688
FOODS_1_002_TX_1_validation         8.921875
FOODS_1_002_TX_2_validation         8.929688
FOODS_1_002_TX_3_validation         8.929688
FOODS_1_002_WI_1_validation         8.929688
FOODS_1_002_WI_2_validation         8.984375
FOODS_1_002_WI_3_validation         8.929688
FOODS_1_003_CA_1_validation         2.972656
FOODS_1_003_CA_2_validation         2.972656
FOODS_1

In [18]:
data_df = data_df.merge(mean_sell_price_df["mean_sell_price"], on="id")

In [19]:
data_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,mean_sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,8.28125
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-30,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,8.28125
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-31,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,8.28125
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-02-01,11101,NaN,NaN,NaN,NaN,1,1,0,NaN,8.28125
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-02-02,11101,NaN,NaN,NaN,NaN,1,0,1,NaN,8.28125


In [20]:
data_df["diff_sell_price"] = data_df["mean_sell_price"] - data_df["sell_price"]

In [21]:
data_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,mean_sell_price,diff_sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,8.28125,NaN
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-30,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,8.28125,NaN
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-31,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,8.28125,NaN
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-02-01,11101,NaN,NaN,NaN,NaN,1,1,0,NaN,8.28125,NaN
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-02-02,11101,NaN,NaN,NaN,NaN,1,0,1,NaN,8.28125,NaN
